<a href="https://colab.research.google.com/github/r-aashish/Fake_News_Thumbnail/blob/main/Translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade googletrans==4.0.0rc1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.2 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17413 sha256=ef76265dc023008b08e334faf71005092a380691a3bfda756c7766416f32e1fe
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans

In [ ]:
import json
import pandas as pd
from googletrans import Translator
from httpx import Timeout

# Load the dataset
input_file = "/content/labeled_data.json"
output_file = "output_multi_languages.json"
column_to_translate = "title"
languages = ["bn", "hi", "te","es","de","zh-TW"]

with open(input_file, "r") as f:
    json_data = json.load(f)

df = pd.DataFrame(json_data)

# Create a Translator object
translator = Translator(service_urls=['translate.google.com'], timeout=Timeout(50))

# Loop through each row in the dataset and translate the specified column to multiple languages
for index, row in df.iterrows():
    for lang in languages:
        try:
            column_name = f"{column_to_translate}_{lang}"
            original_text = row[column_to_translate]
            translated_text = translator.translate(original_text, src="en", dest=lang).text
            df.at[index, column_name] = translated_text
        except:
            continue

# Save the translated dataset to a new JSON file
translated_json_data = df.to_dict(orient="records")
with open(output_file, "w") as f:
    json.dump(translated_json_data, f, ensure_ascii=False, indent=2) 

In [ ]:
import json

# Specify the names of the input JSON files
input_files = ["/content/labeled_data.json","/content/output_bn.json", "/content/output_de.json" , "/content/output_es.json", "/content/output_hi.json", "/content/output_te.json", "/content/output_zh-TW.json"]

combined_data = []

# Loop through each file and read its content
for input_file in input_files:
    with open(input_file, "r") as file:
        data = json.load(file)
        combined_data.extend(data)

# Save the combined data to a single output JSON file
with open("combined_output.json", "w") as output_file:
    json.dump(combined_data, output_file, ensure_ascii=False, indent=2)

In [ ]:
import json
import random

# Load the input JSON file
with open("/content/combined_output.json", "r") as input_file:
    data = json.load(input_file)

# Shuffle the data
random.shuffle(data)

# Calculate the indices for splitting
length = len(data)
index_80 = int(length * 0.8)
index_90 = int(length * 0.9)

# Split the data into train, test, and validation parts
train_data = data[:index_80]
test_data = data[index_80:index_90]
val_data = data[index_90:]

# Save the split data to separate JSON files
with open("Multi_train_data.json", "w") as output_file:
    json.dump(train_data, output_file, ensure_ascii=False, indent=2)

with open("Multi_test_data.json", "w") as output_file:
    json.dump(test_data, output_file, ensure_ascii=False, indent=2)

with open("Multi_val_data.json", "w") as output_file:
    json.dump(val_data, output_file, ensure_ascii=False, indent=2)